In [1]:
import pandas as pd
import matplotlib.pyplot as plt

In [9]:
TENANT_NUM = 2
DURATION = 900
LAT_MU = 3
CACHE_SIZE_RATIO = 0.5

RESULT_DIR = "/home/yunchi/582/resalloc/results/"
ams_df = pd.read_csv(RESULT_DIR + 
    f"lat{LAT_MU}_cr{CACHE_SIZE_RATIO}/selected_data_tenant{TENANT_NUM}_time0-{DURATION}_iter0_AMShare.csv"
)
glob_df = pd.read_csv(RESULT_DIR +
    f"lat{LAT_MU}_cr{CACHE_SIZE_RATIO}/selected_data_tenant{TENANT_NUM}_time0-{DURATION}_iter0_GlobalLRU.csv"
)
maxmin_df = pd.read_csv(RESULT_DIR +
    f"lat{LAT_MU}_cr{CACHE_SIZE_RATIO}/selected_data_tenant{TENANT_NUM}_time0-{DURATION}_iter0_Max-Min.csv"
)
memshare_df = pd.read_csv(RESULT_DIR +
    f"lat{LAT_MU}_cr{CACHE_SIZE_RATIO}/selected_data_tenant{TENANT_NUM}_time0-{DURATION}_iter0_Memshare.csv"
)

In [10]:
tntids = set(ams_df["tntid"])
for base_df in [glob_df, maxmin_df, memshare_df]:
    print("-"*20)
    avg_lat_ams = (ams_df["finish_ts"]-ams_df["issue_ts"]).mean()
    avg_lat_base = (base_df["finish_ts"]-base_df["issue_ts"]).mean()
    print("collective lat_ratio:", avg_lat_ams/avg_lat_base)

    hit_ams = len(ams_df[ams_df["hit"] == True])
    hit_base = len(base_df[base_df["hit"] == True])
    print("collective hitrate_ratio:", hit_ams/hit_base)

    for t in tntids:
        ams_tdf = ams_df[ams_df["tntid"] == t]
        base_tdf = base_df[base_df["tntid"] == t]
        
        avg_lat_ams = (ams_tdf["finish_ts"]-ams_tdf["issue_ts"]).mean()
        avg_lat_base = (base_tdf["finish_ts"]-base_tdf["issue_ts"]).mean()
        print(t, "lat_ratio:", avg_lat_ams/avg_lat_base)
        
        hit_ams = len(ams_tdf[ams_tdf["hit"] == True])
        hit_base = len(base_tdf[base_tdf["hit"] == True])
        print(t, "hitrate_ratio:", hit_ams/hit_base)

--------------------
collective lat_ratio: 0.9722596447582316
collective hitrate_ratio: 1.0518203170874927
2534 lat_ratio: 0.9813337605416346
2534 hitrate_ratio: 1.0368020304568528
4903 lat_ratio: 0.9773816660244659
4903 hitrate_ratio: 1.0448833034111311
5783 lat_ratio: 0.9848279118084923
5783 hitrate_ratio: 1.0288461538461537
3179 lat_ratio: 0.9546880680897128
3179 hitrate_ratio: 1.0684793554884189
2990 lat_ratio: 0.9687461179986963
2990 hitrate_ratio: 1.0635245901639345
1392 lat_ratio: 0.978896080808269
1392 hitrate_ratio: 1.0463458110516934
2480 lat_ratio: 0.9775828688312166
2480 hitrate_ratio: 1.0361111111111112
2263 lat_ratio: 0.9681401477902762
2263 hitrate_ratio: 1.081237911025145
1304 lat_ratio: 0.9708893151312658
1304 hitrate_ratio: 1.0643382352941178
4889 lat_ratio: 0.9659962324930459
4889 hitrate_ratio: 1.054585152838428
--------------------
collective lat_ratio: 0.9678345795561019
collective hitrate_ratio: 1.0646359583952452
2534 lat_ratio: 0.957060761273327
2534 hitrate_ra

In [11]:
avg_lat_ms = (memshare_df["finish_ts"]-memshare_df["issue_ts"]).mean()
avg_lat_base = (glob_df["finish_ts"]-glob_df["issue_ts"]).mean()
print("collective lat_ratio:", avg_lat_ms/avg_lat_base)

hit_ms = len(memshare_df[memshare_df["hit"] == True])
hit_base = len(glob_df[glob_df["hit"] == True])
print("collective hitrate_ratio:", hit_ms/hit_base)

collective lat_ratio: 1.0152704898743894
collective hitrate_ratio: 0.9977980035231944
